In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# New Section

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [3]:
import cv2
import numpy as np
import glob
import shutil
import os

In [4]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [5]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.3/721.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 99.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
# get bounding box from images by using OpenCV
def get_imgage_range(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
  lower = np.array([0, 0, 0], dtype = "uint8")
  upper = np.array([255, 50, 255], dtype = "uint8")
  img = cv2.inRange(img, lower, upper)
  img = cv2.blur(img, (2, 2))
  ret, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY)
  img = cv2.bitwise_not(img)
  contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  contours = max(contours, key=lambda x: cv2.contourArea(x))
  #get bounding box posotion
  xmin, ymin, xmax, ymax = cv2.boundingRect(contours)
  #get the original width,height of the image
  dimensions = img.shape
  h = img.shape[0]
  w = img.shape[1]

  #to calculate the bndBox info of this image for yolo
  xp = (xmin + (xmax-xmin)/2) * 1.0 / w
  yp = (ymin + (ymax-ymin)/2) * 1.0 / h
  wp = (xmax-xmin) * 1.0 / w
  hp = (ymax-ymin) * 1.0 / h
  return xp, yp, wp, hp

In [ ]:
from ultralytics import YOLO
path = '/content/drive/MyDrive/dataset'
model = YOLO("yolov8m.pt")

finalpathtrain = '/content/drive/MyDrive/data/train'
finalpathtest = '/content/drive/MyDrive/data/val'
finalpathtrainimages = finalpathtrain + "/images"
finalpathtrainlabel = finalpathtrain + "/labels"

finalpathtestimages = finalpathtest + "/images"
finalpathtestlabel = finalpathtest + "/labels"

if not os.path.exists(finalpathtrainimages):
         os.mkdir(finalpathtrainimages)
if not os.path.exists(finalpathtrainlabel):
         os.mkdir(finalpathtrainlabel)

if not os.path.exists(finalpathtestimages):
         os.mkdir(finalpathtestimages)
if not os.path.exists(finalpathtestlabel):
         os.mkdir(finalpathtestlabel)

for dir_path in  glob.iglob(path+'/train/*') :
  dirnames=dir_path.split('/')[6]
  print(dir_path)
  #define class for images
  label=0
  if dirnames == 'freshapples':
    label=0
  elif dirnames == 'freshbanana':
    label=1
  elif dirnames == 'freshoranges':
    label=2
  elif dirnames == 'rottenapples':
    label=3
  elif dirnames == 'rottenbanana':
    label=4
  elif dirnames == 'rottenoranges':
    label=5
  for img_path in glob.iglob(dir_path + "/*.png"):
    print(img_path)
    #to generate image's yolo lable
    if os.path.exists(img_path):
      #get yolo label
      xmin, ymin, xmax, ymax=get_imgage_range(img_path)
      out_lbl_text = finalpathtrainlabel + "/" + (img_path.split('/')[7]).split('.png')[0] + ".txt"
      with open(out_lbl_text, 'a+') as f:
        f.write('%s %s %s %s %s\n' % (label,xmin, ymin, xmax, ymax))
      shutil.copy(img_path, finalpathtrainimages + "/" + (img_path.split('/')[7]))





100%|██████████| 49.7M/49.7M [00:00<00:00, 297MB/s]


/content/drive/MyDrive/dataset/train/rottenoranges
/content/drive/MyDrive/dataset/train/rottenoranges/rotated_by_15_Screen Shot 2018-06-12 at 11.19.08 PM.png
/content/drive/MyDrive/dataset/train/rottenoranges/rotated_by_15_Screen Shot 2018-06-12 at 11.18.53 PM.png
/content/drive/MyDrive/dataset/train/rottenoranges/rotated_by_15_Screen Shot 2018-06-12 at 11.18.41 PM.png
/content/drive/MyDrive/dataset/train/rottenoranges/rotated_by_15_Screen Shot 2018-06-12 at 11.19.16 PM.png
/content/drive/MyDrive/dataset/train/rottenoranges/rotated_by_15_Screen Shot 2018-06-12 at 11.21.17 PM.png
/content/drive/MyDrive/dataset/train/rottenoranges/rotated_by_15_Screen Shot 2018-06-12 at 11.19.47 PM.png
/content/drive/MyDrive/dataset/train/rottenoranges/rotated_by_15_Screen Shot 2018-06-12 at 11.23.33 PM.png
/content/drive/MyDrive/dataset/train/rottenoranges/rotated_by_15_Screen Shot 2018-06-12 at 11.24.24 PM.png
/content/drive/MyDrive/dataset/train/rottenoranges/rotated_by_15_Screen Shot 2018-06-12 at 11

In [ ]:
for dir_path in  glob.iglob(path+'/test/*') :
  dirnames=dir_path.split('/')[6]
  print(dir_path)
  #define class for images
  label=0
  if dirnames == 'freshapples':
    label=0
  elif dirnames == 'freshbanana':
    label=1
  elif dirnames == 'freshoranges':
    label=2
  elif dirnames == 'rottenapples':
    label=3
  elif dirnames == 'rottenbanana':
    label=4
  elif dirnames == 'rottenoranges':
    label=5
  for img_path in glob.iglob(dir_path + "/*.png"):
    print(img_path)
    #to generate image's yolo lable
    if os.path.exists(img_path):
      #get yolo label
      xmin, ymin, xmax, ymax=get_imgage_range(img_path)
      out_lbl_text = finalpathtestlabel + "/" + (img_path.split('/')[7]).split('.png')[0] + ".txt"
      with open(out_lbl_text, 'a+') as f:
        f.write('%s %s %s %s %s\n' % (label,xmin, ymin, xmax, ymax))
      shutil.copy(img_path, finalpathtestimages + "/" + (img_path.split('/')[7]))

/content/drive/MyDrive/dataset/test/rottenbanana
/content/drive/MyDrive/dataset/test/rottenbanana/rotated_by_15_Screen Shot 2018-06-12 at 8.53.54 PM.png
/content/drive/MyDrive/dataset/test/rottenbanana/rotated_by_15_Screen Shot 2018-06-12 at 8.48.18 PM.png
/content/drive/MyDrive/dataset/test/rottenbanana/rotated_by_15_Screen Shot 2018-06-12 at 8.47.51 PM.png
/content/drive/MyDrive/dataset/test/rottenbanana/rotated_by_15_Screen Shot 2018-06-12 at 8.51.46 PM.png
/content/drive/MyDrive/dataset/test/rottenbanana/rotated_by_15_Screen Shot 2018-06-12 at 8.54.23 PM.png
/content/drive/MyDrive/dataset/test/rottenbanana/rotated_by_15_Screen Shot 2018-06-12 at 8.49.59 PM.png
/content/drive/MyDrive/dataset/test/rottenbanana/rotated_by_15_Screen Shot 2018-06-12 at 8.52.48 PM.png
/content/drive/MyDrive/dataset/test/rottenbanana/rotated_by_15_Screen Shot 2018-06-12 at 8.53.29 PM.png
/content/drive/MyDrive/dataset/test/rottenbanana/rotated_by_15_Screen Shot 2018-06-12 at 8.51.00 PM.png
/content/drive/

In [ ]:
!pip install ultralytics
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
model.train(data="/content/drive/MyDrive/fruitdetectionyolo.yml", epochs=20)


Ultralytics YOLOv8.0.205 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/fruitdetectionyolo.yml, epochs=20, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscrip

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a01d054bd90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     

In [ ]:
trained_model = YOLO("/content/runs/detect/train/weights/best.pt")
# trained_model.predict("/content/drive/MyDrive/data/train/images/rotated_by_15_Screen Shot 2018-06-07 at 2.15.50 PM.png")
#results = trained_model("/content/drive/MyDrive/data/train/images/rotated_by_15_Screen Shot 2018-06-07 at 2.16.54 PM.png")
#results = trained_model("/content/drive/MyDrive/data/train/images/rotated_by_15_Screen Shot 2018-06-13 at 12.08.48 AM.png")
#results = trained_model("/content/drive/MyDrive/data/train/images/rotated_by_15_Screen Shot 2018-06-07 at 2.45.35 PM.png")
#results = trained_model("/content/drive/MyDrive/data/train/images/rotated_by_15_Screen Shot 2018-06-12 at 9.41.57 PM.png")
#results = trained_model("/content/drive/MyDrive/data/train/images/rotated_by_15_Screen Shot 2018-06-12 at 8.54.58 PM.png")
results = trained_model("/content/drive/MyDrive/data/train/images/rotated_by_15_Screen Shot 2018-06-12 at 9.48.04 PM.png")

print(results)


image 1/1 /content/drive/MyDrive/data/train/images/rotated_by_15_Screen Shot 2018-06-12 at 9.48.04 PM.png: 416x640 1 freshbanana, 9.6ms
Speed: 1.9ms preprocess, 9.6ms inference, 2.1ms postprocess per image at shape (1, 3, 416, 640)


[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'freshapples', 1: 'freshbanana', 2: 'freshoranges', 3: 'rottenapples', 4: 'rottenbanana', 5: 'rottenoranges'}
orig_img: array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
     